In [2]:
import pandas as pd
import math

In [3]:
data = pd.read_csv("clean_csv/akoya.csv")

managers = list(data["manager_short_name"].unique())
managers.pop(1)

current_gw = 2 #current gameweek

In [4]:
managers

['AA', 'SB', 'AC', 'SS', 'WN', 'ES', 'YA1', 'RK', 'YE', 'SL', 'YA', 'ST']

In [5]:
data.head()

,player_id,player_name,position,team,gameweek,manager_name,manager_short_name,manager_team_name,squad_position,bench,...,bonus,bps,influence,creativity,threat,ict_index,total_points,in_dreamteam,photo,badge
0,1.0,Balogun,FWD,ARS,1.0,Ali,AA,WALK WIT ME (C),13.0,bench,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,232223,3
1,2.0,Cédric,DEF,ARS,1.0,transfer market,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,58822,3
2,3.0,M.Elneny,MID,ARS,1.0,transfer market,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,153256,3
3,4.0,Fábio Vieira,MID,ARS,1.0,transfer market,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,438098,3
4,5.0,Gabriel,DEF,ARS,1.0,Sandipan,SB,Sorry Nic Jackson,5.0,fielded,...,0.0,2.0,0.2,0.0,0.0,0.0,1.0,False,226597,3


In [6]:
data.columns

Index(['player_id', 'player_name', 'position', 'team', 'gameweek',
       'manager_name', 'manager_short_name', 'manager_team_name',
       'squad_position', 'bench', 'points', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'total_points', 'in_dreamteam', 'photo', 'badge'],
      dtype='object')

# Points

In [7]:
fielded_points = data[(data["manager_name"]!="transfer market")&(data["bench"]=="fielded")]

grouped = fielded_points.groupby(["manager_short_name","manager_name","gameweek"])["points"].sum()
gameweek_df = grouped.reset_index()
gameweek_df

,manager_short_name,manager_name,gameweek,points
0,AA,Ali,1.0,51.0
1,AC,Aakar,1.0,20.0
2,ES,Ege,1.0,31.0
3,RK,Ruslan,1.0,34.0
4,SB,Sandipan,1.0,32.0
5,SL,Sami,1.0,48.0
6,SS,Santi,1.0,31.0
7,ST,Shrey,1.0,38.0
8,WN,Will,1.0,58.0
9,YA,Youssef,1.0,42.0


In [8]:
def get_ranking(df,bench,goal):
    filtered = df[(df["manager_name"]!="transfer market")&(df["bench"]==bench)]
    ranked_df = filtered.groupby(["manager_short_name","manager_name"])
    return ranked_df[goal].sum().sort_values(ascending=False)

In [9]:
get_ranking(data,"fielded","points")

manager_short_name  manager_name
WN                  Will            58.0
AA                  Ali             51.0
SL                  Sami            48.0
YA                  Youssef         42.0
YE                  Youssef         41.0
ST                  Shrey           38.0
RK                  Ruslan          34.0
YA1                 Yahya           34.0
SB                  Sandipan        32.0
ES                  Ege             31.0
SS                  Santi           31.0
AC                  Aakar           20.0
Name: points, dtype: float64

In [10]:
real_ranking = get_ranking(data,"fielded","points").to_frame()
real_ranking = real_ranking.reset_index().rename(columns={'manager_short_name': 'manager'})
real_ranking.to_csv(f'findings/points/real_ranking.csv', index=False)

In [11]:
real_ranking

,manager,manager_name,points
0,WN,Will,58.0
1,AA,Ali,51.0
2,SL,Sami,48.0
3,YA,Youssef,42.0
4,YE,Youssef,41.0
5,ST,Shrey,38.0
6,RK,Ruslan,34.0
7,YA1,Yahya,34.0
8,SB,Sandipan,32.0
9,ES,Ege,31.0


### League by positions

Make 4 tables, ranking everyone depending on each position (gk, def, mid, fwd)

In [12]:
#GK Table
gk = data[data["position"]=="GK"]
get_ranking(gk,"fielded","points")

manager_short_name  manager_name
WN                  Will            12.0
YE                  Youssef          9.0
SS                  Santi            6.0
AA                  Ali              3.0
YA1                 Yahya            3.0
ES                  Ege              2.0
RK                  Ruslan           2.0
SB                  Sandipan         2.0
YA                  Youssef          2.0
SL                  Sami             1.0
ST                  Shrey            1.0
AC                  Aakar            0.0
Name: points, dtype: float64

In [13]:
#DEF Table
deef = data[data["position"]=="DEF"]
get_ranking(deef,"fielded","points")

manager_short_name  manager_name
AA                  Ali             26.0
YA                  Youssef         22.0
SB                  Sandipan        16.0
WN                  Will            10.0
ST                  Shrey            9.0
YA1                 Yahya            9.0
YE                  Youssef          9.0
ES                  Ege              7.0
RK                  Ruslan           5.0
SL                  Sami             5.0
AC                  Aakar            3.0
SS                  Santi            3.0
Name: points, dtype: float64

In [14]:
#MID Table
mid = data[data["position"]=="MID"]
get_ranking(mid,"fielded","points")

manager_short_name  manager_name
YA1                 Yahya           21.0
SL                  Sami            20.0
AC                  Aakar           17.0
ST                  Shrey           16.0
WN                  Will            16.0
YE                  Youssef         16.0
ES                  Ege             15.0
AA                  Ali             13.0
RK                  Ruslan          13.0
SB                  Sandipan        13.0
SS                  Santi           13.0
YA                  Youssef         10.0
Name: points, dtype: float64

In [15]:
#FWD Table
fwd = data[data["position"]=="FWD"]
get_ranking(fwd,"fielded","points")

manager_short_name  manager_name
SL                  Sami            22.0
WN                  Will            20.0
RK                  Ruslan          14.0
ST                  Shrey           12.0
AA                  Ali              9.0
SS                  Santi            9.0
YA                  Youssef          8.0
ES                  Ege              7.0
YE                  Youssef          7.0
SB                  Sandipan         1.0
YA1                 Yahya            1.0
AC                  Aakar            0.0
Name: points, dtype: float64

In [16]:
gks_df = get_ranking(gk,"fielded","points").to_frame()
def_df = get_ranking(deef,"fielded","points").to_frame()
mid_df = get_ranking(mid,"fielded","points").to_frame()
fwd_df = get_ranking(fwd,"fielded","points").to_frame()

gks_df = gks_df.reset_index().rename(columns={'manager_short_name': 'manager'})
def_df = def_df.reset_index().rename(columns={'manager_short_name': 'manager'})
mid_df = mid_df.reset_index().rename(columns={'manager_short_name': 'manager'})
fwd_df = fwd_df.reset_index().rename(columns={'manager_short_name': 'manager'})

gks_df.to_csv(f'findings/points/gk_ranking.csv', index=False)
def_df.to_csv(f'findings/points/def_ranking.csv', index=False)
mid_df.to_csv(f'findings/points/mid_ranking.csv', index=False)
fwd_df.to_csv(f'findings/points/fwd_ranking.csv', index=False)

### Best gw by position

In [17]:
def get_ranking_gw(df):
    filtered = df[(df["manager_name"]!="transfer market")&(df["bench"]=="fielded")]
    ranked_df = filtered.groupby(["manager_short_name","manager_name","gameweek"])
    return ranked_df["points"].sum().sort_values(ascending=False)

In [18]:
#GK GW Table
get_ranking_gw(gk)

manager_short_name  manager_name  gameweek
WN                  Will          1.0         12.0
YE                  Youssef       1.0          9.0
SS                  Santi         1.0          6.0
AA                  Ali           1.0          3.0
YA1                 Yahya         1.0          3.0
ES                  Ege           1.0          2.0
RK                  Ruslan        1.0          2.0
SB                  Sandipan      1.0          2.0
YA                  Youssef       1.0          2.0
SL                  Sami          1.0          1.0
ST                  Shrey         1.0          1.0
AC                  Aakar         1.0          0.0
Name: points, dtype: float64

In [19]:
#DEF GW Table
get_ranking_gw(deef)

manager_short_name  manager_name  gameweek
AA                  Ali           1.0         26.0
YA                  Youssef       1.0         22.0
SB                  Sandipan      1.0         16.0
WN                  Will          1.0         10.0
ST                  Shrey         1.0          9.0
YA1                 Yahya         1.0          9.0
YE                  Youssef       1.0          9.0
ES                  Ege           1.0          7.0
RK                  Ruslan        1.0          5.0
SL                  Sami          1.0          5.0
AC                  Aakar         1.0          3.0
SS                  Santi         1.0          3.0
Name: points, dtype: float64

In [20]:
#MID GW Table
get_ranking_gw(mid)

manager_short_name  manager_name  gameweek
YA1                 Yahya         1.0         21.0
SL                  Sami          1.0         20.0
AC                  Aakar         1.0         17.0
ST                  Shrey         1.0         16.0
WN                  Will          1.0         16.0
YE                  Youssef       1.0         16.0
ES                  Ege           1.0         15.0
AA                  Ali           1.0         13.0
RK                  Ruslan        1.0         13.0
SB                  Sandipan      1.0         13.0
SS                  Santi         1.0         13.0
YA                  Youssef       1.0         10.0
Name: points, dtype: float64

In [21]:
#FWD GW Table
get_ranking_gw(fwd)

manager_short_name  manager_name  gameweek
SL                  Sami          1.0         22.0
WN                  Will          1.0         20.0
RK                  Ruslan        1.0         14.0
ST                  Shrey         1.0         12.0
AA                  Ali           1.0          9.0
SS                  Santi         1.0          9.0
YA                  Youssef       1.0          8.0
ES                  Ege           1.0          7.0
YE                  Youssef       1.0          7.0
SB                  Sandipan      1.0          1.0
YA1                 Yahya         1.0          1.0
AC                  Aakar         1.0          0.0
Name: points, dtype: float64

In [22]:
gks_df = get_ranking_gw(gk).to_frame()
def_df = get_ranking_gw(deef).to_frame()
mid_df = get_ranking_gw(mid).to_frame()
fwd_df = get_ranking_gw(fwd).to_frame()

gks_df = gks_df.reset_index().rename(columns={'manager_short_name': 'manager'})
def_df = def_df.reset_index().rename(columns={'manager_short_name': 'manager'})
mid_df = mid_df.reset_index().rename(columns={'manager_short_name': 'manager'})
fwd_df = fwd_df.reset_index().rename(columns={'manager_short_name': 'manager'})

gks_df.to_csv(f'findings/points/gw_gk_ranking.csv', index=False)
def_df.to_csv(f'findings/points/gw_def_ranking.csv', index=False)
mid_df.to_csv(f'findings/points/gw_mid_ranking.csv', index=False)
fwd_df.to_csv(f'findings/points/gw_fwd_ranking.csv', index=False)

### Bench FC


In [23]:
get_ranking(data,"bench","points")

manager_short_name  manager_name
RK                  Ruslan          7.0
ST                  Shrey           6.0
WN                  Will            6.0
SB                  Sandipan        5.0
ES                  Ege             3.0
YE                  Youssef         3.0
SS                  Santi           2.0
YA1                 Yahya           2.0
YA                  Youssef         1.0
AA                  Ali             0.0
AC                  Aakar           0.0
SL                  Sami            0.0
Name: points, dtype: float64

In [24]:
bench = get_ranking(data,"bench","points").to_frame()

bench = bench.reset_index().rename(columns={'manager_short_name': 'manager'})

bench_total = bench.groupby(["manager","manager_name"]).sum().sort_values("points",ascending=False)
bench_total = bench_total.reset_index()

bench.to_csv(f'findings/points/bench.csv', index=False)

### Optimised Bench

In [25]:
def optimise(gw,manager):
    benched = data[(data["gameweek"] == gw)&(data["manager_short_name"] == manager)&(data["bench"] == "bench")]
    fielded = data[(data["gameweek"] == gw)&(data["manager_short_name"] == manager)&(data["bench"] == "fielded")]

    points = 0

    positions = [("DEF",3),("MID",2),("FWD",1)]

    b_gk = benched[benched["position"] == "GK"]
    f_gk = fielded[fielded["position"] == "GK"]

    if  f_gk.loc[f_gk.index.tolist()[0],"points"] < b_gk.loc[b_gk.index.tolist()[0],"points"]:
        points += b_gk["points"].item() - f_gk["points"].item()
        benched = benched.drop(index=b_gk.index)

    for indx, b_player in benched.iterrows():
        for pos in positions:
            for indx2, f_player in fielded.iterrows():
                if len(fielded[fielded["position"] == pos[0]]) > pos[1]:
                    if f_player["points"] < b_player["points"]:
                        points += b_player["points"] - f_player["points"]
                        benched = benched.drop(index=indx)
                        fielded = fielded.drop(index=indx2)
                        break  # exit the inner loop and go to the next b_player
            else:
                continue  # only executed if the inner loop didn't break
            break  # exit the outer loop and go to the next b_player

    return points

In [26]:
missed_points = pd.DataFrame(columns=["manager","gameweek","missed_pts"])

for manager in managers:
    for gw in range(1,current_gw+1):
        new_row = {"manager":manager,"gameweek":gw,"missed_pts":optimise(gw,manager)}
        missed_points = missed_points.append(new_row, ignore_index=True)

C:\Users\santi\AppData\Local\Temp\ipykernel_6964\2688571503.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  missed_points = missed_points.append(new_row, ignore_index=True)


IndexError: list index out of range

In [ ]:
missed_points

,manager,gameweek,missed_pts
0,AA,1,0
1,SB,1,2.0
2,AC,1,0
3,SS,1,1.0
4,WN,1,1.0
5,ES,1,1.0
6,YA1,1,1.0
7,RK,1,6.0
8,YE,1,1.0
9,SL,1,1.0


In [ ]:
missed_points_grouped = missed_points.groupby(missed_points["manager"])
missed_points_grouped["missed_pts"].sum().sort_values(ascending=False)

manager
RK     6.0
ST     5.0
SB     2.0
ES     1.0
SL     1.0
SS     1.0
WN     1.0
YA1    1.0
YE     1.0
AA       0
AC       0
YA       0
Name: missed_pts, dtype: object

In [27]:
bench_best = missed_points_grouped["missed_pts"].sum().sort_values(ascending=False).to_frame()
bench_best = bench_best.reset_index().rename(columns={'missed_pts': 'points'})
bench_best.to_csv(f'findings/points/bench_best.csv', index=False)

NameError: name 'missed_points_grouped' is not defined

### GW Podiums

In [28]:
gw_podiums = pd.DataFrame(columns=["manager","manager_name","gameweek","points"])

gw_podiums["manager"] = gameweek_df["manager_short_name"]
gw_podiums["manager_name"] = gameweek_df["manager_name"]
gw_podiums["gameweek"] = gameweek_df["gameweek"]
gw_podiums["points"] = gameweek_df["points"]
gw_podiums["rank"] = gw_podiums.groupby("gameweek")["points"].rank(ascending=False).apply(math.floor)
gw_podiums["podium"] = gw_podiums["rank"]<4


gw_podiums

,manager,manager_name,gameweek,points,rank,podium
0,AA,Ali,1.0,51.0,2,True
1,AC,Aakar,1.0,20.0,12,False
2,ES,Ege,1.0,31.0,10,False
3,RK,Ruslan,1.0,34.0,7,False
4,SB,Sandipan,1.0,32.0,9,False
5,SL,Sami,1.0,48.0,3,True
6,SS,Santi,1.0,31.0,10,False
7,ST,Shrey,1.0,38.0,6,False
8,WN,Will,1.0,58.0,1,True
9,YA,Youssef,1.0,42.0,4,False


In [29]:
gw_podiums.groupby(["manager"])["rank"].mean().sort_values()

manager
WN      1.0
AA      2.0
SL      3.0
YA      4.0
YE      5.0
ST      6.0
RK      7.0
YA1     7.0
SB      9.0
ES     10.0
SS     10.0
AC     12.0
Name: rank, dtype: float64

In [39]:
total_podiums_df = pd.DataFrame(index=['1st', '2nd', '3rd'], columns=gw_podiums["manager"].unique())

grouped = gw_podiums.groupby(["manager", 'rank']).size()

# loop through each manager and count their 1st, 2nd, and 3rd place finishes
for manager in total_podiums_df.columns:
    try:
        total_podiums_df.loc['1st', manager] = grouped[manager, 1]
    except KeyError:
        total_podiums_df.loc['1st', manager] = 0
    try:
        total_podiums_df.loc['2nd', manager] = grouped[manager, 2]
    except KeyError:
        total_podiums_df.loc['2nd', manager] = 0
    try:
        total_podiums_df.loc['3rd', manager] = grouped[manager, 3]
    except KeyError:
        total_podiums_df.loc['3rd', manager] = 0

total_podiums_df.loc['Total'] = total_podiums_df.sum(axis=0)

total_podiums_df = total_podiums_df.transpose()
total_podiums_df = total_podiums_df.reset_index().rename(columns={"index":"manager"})
total_podiums_df.merge(gw_podiums[["manager","manager_name"]],left_on="manager",right_on="manager")

,manager,1st,2nd,3rd,Total,manager_name
0,AA,0,1,0,1,Ali
1,AC,0,0,0,0,Aakar
2,ES,0,0,0,0,Ege
3,RK,0,0,0,0,Ruslan
4,SB,0,0,0,0,Sandipan
5,SL,0,0,1,1,Sami
6,SS,0,0,0,0,Santi
7,ST,0,0,0,0,Shrey
8,WN,1,0,0,1,Will
9,YA,0,0,0,0,Youssef


In [33]:
total_podiums_df.columns

Index(['1st', '2nd', '3rd', 'Total'], dtype='object')

In [49]:
total_podiums_df.to_csv(f'findings/points/podiums.csv', index=True,index_label="manager")

### Tottenham Award

Most GW without podium

In [50]:
def longest_streak(df,who):
    current_streak = 1
    max_streak = 1
    indx = 0
    
    df = df[df["manager"]==who]
    
    series = df["podium"]

    for i in range(1, len(series)):
        if series.iloc[i]:
            max_streak = max(max_streak, current_streak)
            current_streak = 1
        else:
            current_streak += 1
            if current_streak > max_streak:
                indx = i

    return max(max_streak, current_streak),df.iloc[indx]["manager"]

In [51]:
longest_nopodium_streak = (0,"")
tottenham = pd.DataFrame(columns=["streak_length"])

for manager in managers:
    streak = longest_streak(gw_podiums, manager)
    if longest_nopodium_streak[0] < streak[0]:
        longest_nopodium_streak = streak
    tottenham.loc[manager,"streak_length"] = streak[0]
    

longest_nopodium_streak

(1, 'AA')

In [52]:
tottenham = tottenham.sort_values(by="streak_length",ascending=False)

In [53]:
tottenham = tottenham.reset_index().rename(columns={'index': 'manager'})

tottenham.to_csv(f'findings/points/tottenham.csv', index=False)

In [54]:
tottenham

,manager,streak_length
0,AA,1
1,SB,1
2,AC,1
3,SS,1
4,WN,1
5,ES,1
6,YA1,1
7,RK,1
8,YE,1
9,SL,1


### GW Losers

In [46]:
last_df = pd.DataFrame(index=["Last"], columns=gw_podiums["manager"].unique())

last_grouped = gw_podiums.groupby(["manager", 'rank']).size()

# loop through each manager and count their last place finishes
for manager in last_df.columns:
    try:
        last_df.loc['Last', manager] = last_grouped[manager, len(last_df.columns)]
    except KeyError:
        last_df.loc['1st', manager] = 0


last_df = last_df.transpose().sort_values(by="Last",ascending=False)

In [47]:
last_df

,Last,1st
AC,1,NaN
AA,NaN,0
ES,NaN,0
RK,NaN,0
SB,NaN,0
SL,NaN,0
SS,NaN,0
ST,NaN,0
WN,NaN,0
YA,NaN,0


In [57]:
last_df = last_df.reset_index().rename(columns={'index': 'manager'})

last_df.to_csv(f'findings/points/last_df.csv', index=False)

# Players

### 1-team Players

In [58]:
def get_loyalty(data):
    grouped = data.groupby(["manager_short_name","manager_name","player_name","team","photo"]).agg({'player_id': 'size', 'points': 'sum'}).sort_values('player_id',ascending=False)
    return grouped

In [59]:
#most owned players per club
no_tm = data[(data["manager_name"]!="transfer market")]
loyalty_df = get_loyalty(no_tm)
loyalty_df=loyalty_df.drop(columns="points")
loyalty_df

player_id
manager_short_name manager_name player_name team photo            
AA                 Ali          A.Becker    LIV  116535          1
ST                 Shrey        Kilman      WOL  214048          1
                                Martinez    AVL  98980           1
                                Mings       AVL  149484          1
                                Ream        FUL  82514           1
...                                                            ...
SB                 Sandipan     Bowen       WHU  178186          1
                                Colwill     CHE  460028          1
                                Danjuma     EVE  220307          1
                                Estupiñan   BHA  204214          1
YE                 Youssef      White       ARS  198869          1

[180 rows x 1 columns]

In [61]:
loyalty_df.loc["YA"]

player_id
manager_name player_name  team photo            
Youssef      Antonio      WHU  57531           1
             Casemiro     MUN  61256           1
             Chalobah     CHE  180736          1
             Raya         BRE  154561          1
             Darwin       LIV  447203          1
             João Pedro   BHA  475168          1
             Konaté       LIV  204716          1
             L.Paquetá    WHU  224024          1
             Mac Allister LIV  243016          1
             Malacia      MUN  222690          1
             R.Varane     MUN  90152           1
             Rashford     MUN  176297          1
             Rice         ARS  204480          1
             Sanchez      CHE  215059          1
             Walker       MCI  58621           1

In [62]:
#most fielded players per club
most_played_df = get_loyalty(fielded_points)
most_played_df

player_id  points
manager_short_name manager_name player_name team photo                    
AA                 Ali          A.Becker    LIV  116535          1     3.0
ST                 Shrey        Maddison    TOT  172780          1     9.0
WN                 Will         Robertson   LIV  122798          1     2.0
                                Nketiah     ARS  205533          1     8.0
                                Martinelli  ARS  444145          1     5.0
...                                                            ...     ...
RK                 Ruslan       José Sá     WOL  149065          1     2.0
                                Ings        WHU  84939           1     1.0
                                Haaland     MCI  223094          1    13.0
                                Dunk        BHA  83299           1     2.0
YE                 Youssef      White       ARS  198869          1     1.0

[132 rows x 2 columns]

In [63]:
loyalty = loyalty_df.reset_index().rename(columns={'manager_short_name': 'manager'})
most_played = most_played_df.reset_index().rename(columns={'manager_name': 'manager'})
most_played["ppg"] = round(most_played["points"]/most_played["player_id"],2)

loyalty.to_csv(f'findings/players/loyalty.csv', index=False)
most_played.to_csv(f'findings/players/most_played.csv', index=False)

Players in the most teams

In [64]:
most_teams_grouped = loyalty_df.groupby(["player_name","team","photo"]).size().sort_values(axis=0,ascending=False)
most_teams_df = most_teams_grouped.to_frame().sort_values(by=[0, 'player_name'], ascending=[False, True])
most_teams_df[most_teams_df.iloc[:,0]>3]

,,,0
player_name,team,photo,


In [65]:
most_teams_df.columns

RangeIndex(start=0, stop=1, step=1)

In [66]:
most_teams_df = most_teams_df.reset_index()

most_teams_df.to_csv(f'findings/players/most_teams.csv', index=False)

### Club Mascot

In [67]:
def get_clubmascot(data):
    mascot_grouped = data.groupby(["manager_short_name","manager_name","team"]).sum().sort_values(by=["player_id", 'manager_short_name'], ascending=[False, True])
    return mascot_grouped

In [68]:
#most owned club
club_mascot = get_clubmascot(most_played_df)
club_mascot["bench"] = get_clubmascot(loyalty_df)
club_mascot.rename(columns={"player_id": 'fielded'}, inplace=True)
club_mascot["ratio"] = round(club_mascot["fielded"]/club_mascot["bench"],2)
club_mascot["ppg"] = round(club_mascot["points"]/club_mascot["fielded"],2)
club_mascot[club_mascot["fielded"]>15].sort_values(by="points",ascending=False)[:20]

,,,fielded,points,bench,ratio,ppg
manager_short_name,manager_name,team,,,,,


In [69]:
club_mascot = club_mascot.reset_index().rename(columns={'manager_short_name': 'manager'})

club_mascot.to_csv(f'findings/players/club_mascot.csv', index=False)

# Stats

### Most Goals

In [44]:
stats = get_ranking(data,"fielded","goals_scored").to_frame()
bench_stats = get_ranking(data,"bench","goals_scored").to_frame()

### Most Assists

In [47]:
assists = get_ranking(data,"fielded","assists").to_frame()
assists_bench = get_ranking(data,"bench","assists").to_frame()
stats["assists"] = assists["assists"]
bench_stats["assists_bench"] = assists_bench["assists"]

### Most Clean Sheets

In [48]:
gk_def = data[(data["position"]=="GK")|(data["position"]=="DEF")]

clean_sheets = get_ranking(gk_def,"fielded","clean_sheets").to_frame()
clean_sheets_bench = get_ranking(gk_def,"bench","clean_sheets").to_frame()
stats["clean_sheets"] = clean_sheets["clean_sheets"]
bench_stats["clean_sheets_bench"] = clean_sheets_bench["clean_sheets"]

### Avg Minutes Played

In [71]:
minutes = round(get_ranking(data,"fielded","minutes").to_frame()/11)
stats["minutes"] = minutes["minutes"]

In [82]:
type(get_ranking(data,"fielded","minutes").to_frame()["minutes"][0])

numpy.float64

In [74]:
round(get_ranking(data,"fielded","minutes").to_frame()/11)

,,minutes
manager_short_name,manager_name,
WN,Will,84.0
YA,Youssef,83.0
ST,Shrey,76.0
YE,Youssef,73.0
ES,Ege,72.0
SL,Sami,70.0
AA,Ali,69.0
SB,Sandipan,68.0
YA1,Yahya,67.0


In [72]:
minutes

,,minutes
manager_short_name,manager_name,
WN,Will,84.0
YA,Youssef,83.0
ST,Shrey,76.0
YE,Youssef,73.0
ES,Ege,72.0
SL,Sami,70.0
AA,Ali,69.0
SB,Sandipan,68.0
YA1,Yahya,67.0


### Maguire Award
Most Goals Conceded

In [50]:
goals_conceded = get_ranking(gk_def,"fielded","goals_conceded").to_frame()
goals_conceded_bench = get_ranking(gk_def,"bench","goals_conceded").to_frame()
stats["goals_conceded"] = goals_conceded["goals_conceded"]
bench_stats["goals_conceded_bench"] = goals_conceded_bench["goals_conceded"]

### 2016 Pessi Award
Most Missed Pens

In [51]:
penalties = get_ranking(data,"fielded","penalties_missed").to_frame()
saved_penalties = get_ranking(data,"fielded","penalties_saved").to_frame()
stats["penalties_missed"] = penalties["penalties_missed"]
stats["penalties_saved"] = saved_penalties["penalties_saved"]

### Not De Gea Award
Most Pens Saved

In [52]:
get_ranking(data,"fielded","penalties_saved")

manager_short_name  manager_name
AA                  Ali             0.0
AC                  Aakar           0.0
ES                  Ege             0.0
RK                  Ruslan          0.0
SB                  Sandipan        0.0
SL                  Sami            0.0
SS                  Santi           0.0
ST                  Shrey           0.0
WN                  Will            0.0
YA                  Youssef         0.0
YA1                 Yahya           0.0
YE                  Youssef         0.0
Name: penalties_saved, dtype: float64

### Prime Maguire Award
Most Own Goals

In [53]:
own_goals = get_ranking(data,"fielded","own_goals").to_frame()
stats["own_goals"] = own_goals["own_goals"]

### Sergio Ramos Award
Most Red Cards

In [54]:
red_cards = get_ranking(data,"fielded","red_cards").to_frame()
stats["red_cards"] = red_cards["red_cards"]

### Sergio Ramos Lite Award
Most Yellow Cards

In [55]:
yellow_cards = get_ranking(data,"fielded","yellow_cards").to_frame()
stats["yellow_cards"] = yellow_cards["yellow_cards"]

### The Wall Award
Most Saves

In [56]:
saves = get_ranking(data,"fielded","saves").to_frame()
stats["saves"] = saves["saves"]

### BPS Merchant

In [91]:
data

,player_id,player_name,position,team,gameweek,manager_name,manager_short_name,manager_team_name,squad_position,bench,...,bonus,bps,influence,creativity,threat,ict_index,total_points,in_dreamteam,photo,badge
0,1.0,Balogun,FWD,ARS,1.0,Ali,AA,WALK WIT ME (C),13.0,bench,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,232223,3
1,2.0,Cédric,DEF,ARS,1.0,transfer market,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,58822,3
2,3.0,M.Elneny,MID,ARS,1.0,transfer market,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,153256,3
3,4.0,Fábio Vieira,MID,ARS,1.0,transfer market,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,438098,3
4,5.0,Gabriel,DEF,ARS,1.0,Sandipan,SB,Sorry Nic Jackson,5.0,fielded,...,0.0,2.0,0.2,0.0,0.0,0.0,1.0,False,226597,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
658,572.0,N.Semedo,DEF,WOL,1.0,transfer market,NaN,NaN,NaN,NaN,...,0.0,17.0,27.4,17.3,2.0,4.7,2.0,False,200402,39
659,573.0,Toti,DEF,WOL,1.0,transfer market,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,510362,39
660,574.0,Boubacar Traore,MID,WOL,1.0,transfer market,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,476502,39
661,590.0,Cunha,FWD,WOL,1.0,Youssef,YE,Beanus Hurty F.C,10.0,fielded,...,0.0,3.0,6.2,11.7,24.0,4.2,2.0,False,430871,39


In [92]:
get_ranking(data,"fielded","bonus")

manager_short_name  manager_name
SL                  Sami            8.0
AA                  Ali             7.0
SB                  Sandipan        6.0
WN                  Will            6.0
RK                  Ruslan          4.0
ST                  Shrey           4.0
YA1                 Yahya           4.0
YA                  Youssef         3.0
AC                  Aakar           2.0
SS                  Santi           2.0
YE                  Youssef         2.0
ES                  Ege             0.0
Name: bonus, dtype: float64

NameError: name 'gameweeks_df' is not defined

In [60]:
filtered_bps = data[(data["manager_name"]!="transfer market")&(data["bench"]=="fielded")]
df = filtered_bps.groupby(["manager_short_name","manager_name"])
bonus = df["bonus"].sum().sort_values(ascending=False).to_frame()
stats["bonus"] = bonus["bonus"]

In [90]:
bonus

,,bonus
manager_short_name,manager_name,
SL,Sami,8.0
AA,Ali,7.0
SB,Sandipan,6.0
WN,Will,6.0
RK,Ruslan,4.0
ST,Shrey,4.0
YA1,Yahya,4.0
YA,Youssef,3.0
AC,Aakar,2.0


In [65]:
stats

,,goals_scored,assists,clean_sheets,minutes,goals_conceded,penalties_missed,penalties_saved,own_goals,red_cards,yellow_cards,saves,bonus,dreamteam
manager_short_name,manager_name,,,,,,,,,,,,,
SL,Sami,4.0,1.0,1.0,35.0,8.0,0.0,0.0,0.0,0.0,3.0,4.0,8.0,3
WN,Will,3.0,2.0,2.0,42.0,2.0,0.0,0.0,0.0,0.0,1.0,9.0,6.0,1
AA,Ali,2.0,1.0,3.0,35.0,6.0,0.0,0.0,0.0,0.0,1.0,4.0,7.0,1
ES,Ege,2.0,0.0,1.0,36.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0
RK,Ruslan,2.0,1.0,0.0,33.0,4.0,0.0,0.0,0.0,0.0,1.0,2.0,4.0,1
SS,Santi,2.0,0.0,1.0,32.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0
YA,Youssef,2.0,0.0,2.0,42.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,1
YE,Youssef,2.0,2.0,1.0,36.0,5.0,0.0,0.0,0.0,0.0,1.0,6.0,2.0,0
AC,Aakar,1.0,1.0,0.0,18.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,1


### TOTW Merchant

In [62]:
dreamteam = get_ranking(data,"fielded","in_dreamteam").to_frame()
stats["dreamteam"] = dreamteam["in_dreamteam"]

In [ ]:
stats.to_csv("findings/stats/stats.csv")
bench_stats.to_csv("findings/stats/bench_stats.csv")

# Transfers

In [ ]:
transfers = pd.read_csv("clean_csv/transfers.csv")

In [ ]:
transfers[:10]

,players_id_in,players_id_out,manager_id,gameweek,type,result,transfer_id,manager_short_name,player_name_in,photo_in,team_in,badge_in,player_name_out,photo_out,team_out,badge_out
0,616,538,210440,1,w,a,289182,SB,Højlund,497894,MUN,1,Mubama,487837,WHU,21
1,615,611,151920,1,w,a,504503,AA,Gvardiol,477424,MCI,43,Disasi,220362,CHE,8
2,616,13,175200,1,w,di,627373,WN,Højlund,497894,MUN,1,Nketiah,205533,ARS,3
3,616,485,154164,1,w,di,770326,SL,Højlund,497894,MUN,1,McBurnie,169432,SHU,49
4,288,35,154164,1,w,a,770327,SL,Wilson,153682,FUL,54,Buendia,195546,AVL,7
5,63,35,154164,1,w,do,770328,SL,Billing,168991,BOU,91,Buendia,195546,AVL,7
6,177,541,178696,1,f,a,927658,ES,Rodriguez,44683,BUR,90,Scamacca,195899,WHU,21
7,64,557,154164,1,f,a,938231,SL,Brooks,111317,BOU,91,Hee Chan,184754,WOL,39


In [ ]:
def get_ranking_transfer(df,types):
    if types == "all":
        filtered = df
    else:
        filtered = df[(df["type"]==types)]
    ranked_df = filtered.groupby(["manager_short_name","manager_name"])
    return ranked_df["transfer_id"].size().sort_values(ascending=False)

### Ranking by Total Transfers

In [ ]:
accepted = transfers[transfers["result"]=="a"].reset_index()

In [ ]:
num_transfers = get_ranking_transfer(accepted,"all").to_frame()
num_transfers["waivers"] = get_ranking_transfer(accepted,"w").to_frame()
num_transfers["free_agents"] = get_ranking_transfer(accepted,"f").to_frame()
num_transfers.loc["YE","free_agents"] = 0

In [ ]:
num_transfers.reset_index().rename(columns={'manager_short_name': 'manager'}).to_csv("findings/transfers/num_transfers.csv")

### Trade Winner & Loser


In [ ]:
#get how long a transfer stays at the club, and how long until someone signs him up again
for indx, row in accepted.iterrows():
    player_in = row["players_id_in"]
    player_out = row["players_id_out"]

    for i in range(indx+1, len(accepted)):
        if player_out == accepted.loc[i, "players_id_in"]:
            accepted.loc[indx, "gw_until_in"] = accepted.loc[i, "gameweek"]-accepted.loc[indx, "gameweek"] #gameweeks until someone else signs this player
            accepted.loc[i, "past_manager"] = accepted.loc[indx, "manager_short_name"] #who owned him before
            accepted.loc[indx, "next_manager"] = accepted.loc[i, "manager_short_name"] #who will own him next
            break
    
    for i in range(indx+1, len(accepted)):
        if player_in == accepted.loc[i, "players_id_out"]:
            accepted.loc[indx, "gw_until_out"] = accepted.loc[i, "gameweek"]-accepted.loc[indx, "gameweek"] #gameweeks until this player leaves the team
            break
        

In [ ]:
def get_points(player_id,gw):
    return data[(data["player_id"]==player_id)&(data["gameweek"]==gw)]["points"].iloc[0]

In [ ]:
for indx, row in accepted.iterrows():
    gw = row["gameweek"]
    gws_out = row["gw_until_out"]
    player_in = row["players_id_in"]
    player_out = row["players_id_out"]

    points_in = 0
    points_out = 0

    
    if gws_out >= 3:
        for i in range(1,4):
            x = i - 1
            points_in_gw = get_points(player_in,gw + x)
            points_out_gw = get_points(player_out,gw + x)

            points_in += points_in_gw
            points_out += points_out_gw
    elif math.isnan(gws_out) == False:
        for i in range(1,int(gws_out+1)):
            x = i - 1
            points_in_gw = get_points(player_in,gw + x)
            points_out_gw = get_points(player_out,gw + x)

            points_in += points_in_gw
            points_out += points_out_gw

    accepted.loc[indx,"in_points"] = points_in
    accepted.loc[indx,"out_points"] = points_out
    accepted.loc[indx,"net_points"] = points_in - points_out

KeyError: 'gw_until_out'

In [ ]:
new_order = ['index', 'players_id_in', 'player_name_in', 'photo_in','team_in', 'in_points', 'past_manager' , 'gw_until_out', 'manager_name',
            'manager_short_name', 'gameweek', 'type', 'result', 'transfer_id', 'players_id_out',
            'player_name_out', 'team_out', 'out_points', "next_manager", 'gw_until_in', 'net_points']
accepted = accepted.reindex(columns=new_order)
accepted

,index,players_id_in,player_name_in,photo_in,team_in,in_points,past_manager,gw_until_out,manager_name,manager_short_name,...,type,result,transfer_id,players_id_out,player_name_out,team_out,out_points,next_manager,gw_until_in,net_points
0,0,616,Højlund,497894,MUN,NaN,NaN,NaN,NaN,NaN,...,w,a,289182,538,Mubama,WHU,NaN,NaN,NaN,NaN
1,1,615,Gvardiol,477424,MCI,NaN,NaN,NaN,NaN,NaN,...,w,a,504503,611,Disasi,CHE,NaN,NaN,NaN,NaN
2,4,288,Wilson,153682,FUL,NaN,NaN,NaN,NaN,NaN,...,w,a,770327,35,Buendia,AVL,NaN,NaN,NaN,NaN
3,6,177,Rodriguez,44683,BUR,NaN,NaN,NaN,NaN,NaN,...,f,a,927658,541,Scamacca,WHU,NaN,NaN,NaN,NaN
4,7,64,Brooks,111317,BOU,NaN,NaN,NaN,NaN,NaN,...,f,a,938231,557,Hee Chan,WOL,NaN,NaN,NaN,NaN


In [ ]:
def get_transfer(data):
    filtering = data[["gameweek",'manager_short_name',"net_points","player_name_in","photo_in","team_in","player_name_out","team_out","in_points","out_points","gw_until_out"]]
    return filtering.sort_values(by=["net_points", 'manager_short_name'], ascending=[False, True])

In [ ]:
transfers_net_points = get_transfer(accepted)

In [ ]:
transfers_net_points.rename(columns={"manager_short_name":"manager","player_name_in":"player_name","photo_in":"photo"}).to_csv("findings/transfers/transfers_net_points.csv")

### Most times traded in

In [ ]:
most_in = transfers_net_points.groupby(["manager_short_name","manager_name","player_name_in","photo_in"]).size().sort_values(ascending=False).to_frame()

In [ ]:
most_in

,,,0
manager_short_name,player_name_in,photo_in,


In [ ]:
most_in.reset_index().rename(columns={"manager_short_name":"manager","player_name_in":"player_name","photo_in":"photo",0:"player_id"}).to_csv("findings/transfers/most_in.csv")